In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#Read the csv File
df = pd.read_csv('C:/Users/reraveen/Jupyter/Dataset/ORMB_Data_version.csv',na_values ='')
dfprocessed =df.groupby(['PERSON_ID', 'PRICEITEM_CD']).size().reset_index().rename(columns={0:'Score'})
train=dfprocessed.sample(frac=0.60,random_state=123) #random state is a seed value
test_Actual=dfprocessed.drop(train.index)
print(train.shape)
test = test_Actual.copy()
print(test.shape)
#Convert already bought product score to 0 for testing purpose
test.loc[:, 'Score'] =0
testDataset = pd.concat([train,test], ignore_index=True)
print(testDataset.shape)
TableCustItem =testDataset.pivot(index='PERSON_ID', columns='PRICEITEM_CD', values='Score')
TableCustItem =TableCustItem.replace(np.nan,0)
print(TableCustItem.shape)

df_crossTabedold = pd.crosstab(df.PERSON_ID, df.PRICEITEM_CD).reset_index().rename_axis('',axis='columns') 
df_crossTabedold.shape
TableCustItem =TableCustItem.merge(df_crossTabedold["PERSON_ID"], on='PERSON_ID', how='left')
df_crossTabed=TableCustItem
df_crossTabed1 = df_crossTabed.drop('PERSON_ID', 1)
df_crossTabed1.head()
df_crossTabed1.shape

magnitude = np.sqrt(np.square(df_crossTabed1).sum(axis=1))
df_crossTabed2 = df_crossTabed1.divide(magnitude, axis='index')
df_crossTabed2 =df_crossTabed2.replace(np.nan,0)

def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

data_matrix = calculate_similarity(df_crossTabed2)
data_matrix=data_matrix.replace(np.nan,0)
scoredf = data_matrix.dot(df_crossTabed2.T)
scoredf =scoredf.T
scoredf=scoredf.div(data_matrix.sum(axis=1))
scoredf=scoredf.replace(np.nan,0)
Bought_df=df_crossTabed2.where(df_crossTabed2==0,-999999)
recommd_df=Bought_df.where(Bought_df != 0,scoredf)
top_n = 7
df_final = pd.DataFrame({n: recommd_df.T[col].nlargest(top_n).index.tolist() 
                for n, col in enumerate(recommd_df.T)}).T

new_col =df_crossTabed['PERSON_ID']
df_final.insert(0,'PERSON_ID',new_col,True)
df_final

test_Actual_cross = pd.crosstab(test_Actual.PERSON_ID, test_Actual.PRICEITEM_CD).reset_index().rename_axis('',axis='columns') 
test_Actual_cross

testPersonId=test_Actual_cross['PERSON_ID']
test_Actual_cross=test_Actual_cross.drop(['PERSON_ID'],axis=1)



test_Actual_cross_df=test_Actual_cross.where(test_Actual_cross==0,-999999)
test_Actual_cross_df

Actual_BoughtList = (test_Actual_cross_df == -999999).sum(axis=1)
Actual_BoughtList

top_n = 7
Test_final = pd.DataFrame({n:test_Actual_cross.T[col].nlargest(top_n).index.tolist() 
                for n, col in enumerate(test_Actual_cross.T)}).T

Test_final.insert(0,'PERSON_ID',testPersonId,True)
MeredDf = Test_final.merge(df_final, on="PERSON_ID", how = 'inner')
MeredDf_Selected = MeredDf.loc[:,MeredDf.columns != 'PERSON_ID']
MeredDf['num_uniq'] = [len(set(v[pd.notna(v)].tolist())) for v in MeredDf_Selected.values]
MeredDf['num_uniq']=14-MeredDf['num_uniq']
MeredDf['precision']=MeredDf['num_uniq']/7
meanprecision = MeredDf['precision'].mean()
print("Precision :"+str(meanprecision))
rowCount =len(Test_final.index)
MeredDf['ActualBought']=Actual_BoughtList
actalBoughtCommon=[]
for j in range(0,rowCount):
    elements=[]
    actalCommon=0
    actualBought = Actual_BoughtList[j]
    eval_Value =7
    startindex = 7+1
    endInx =eval_Value*2+1
    elements.extend(Test_final.iloc[j,1:actualBought+1])
    elements.extend(MeredDf.iloc[j,startindex:endInx])
    a_set = set(elements)
    number_unique = len(a_set)
    actalCommon = (actualBought+eval_Value)-number_unique
    actalBoughtCommon.append(actalCommon)
actalBoughtCommon

MeredDf['ActualBoughtPredicted']=actalBoughtCommon
MeredDf['Recall']=MeredDf['ActualBoughtPredicted']/MeredDf['num_uniq']
meanrecall=MeredDf['Recall'].mean()
print("Recall :"+str(meanrecall))
mean_f1score = 2*(meanrecall*meanprecision)/(meanrecall+meanprecision)
print("F1-Score:"+str(mean_f1score))

(105, 3)
(70, 3)
(175, 3)
(40, 84)
Precision :0.7438423645320197
Recall :0.27504105090311987
F1-Score:0.401590962317851


In [ ]:

new_col =df_crossTabed['PERSON_ID']
recommd_df.insert(0,'PERSON_ID',new_col,True)
recommd_df

In [ ]:
top_n = 10
pd.DataFrame({n: recommd_df.T[col].nlargest(top_n).index.tolist() 
    for n, col in enumerate(recommd_df.T)}).T

In [ ]:
(pd.melt(recommd_df.reset_index(), var_name='Predicted_Score', value_name='Items', id_vars='PERSON_ID')
          .set_index('PERSON_ID'))

In [ ]:
print (recommd_df.stack().reset_index(level=1, name='Items')
                 .rename(columns={'level_1':'Predicted_Score'})[['Items','Predicted_Score']])

In [ ]:
#Recommendation Function
def recommended_product(CustomerNo,Nproducts,recommd_df):
    data_items = recommd_df.drop('PERSON_ID', 1)
    user_index = recommd_df[recommd_df.PERSON_ID == CustomerNo].index.tolist()[0]
    rating = data_items.loc[user_index] 
    recommend_products=rating.nlargest(Nproducts)
    #print(recommend_products.index)
    #print(recommend_products.values)
    return  recommend_products

In [ ]:
#Recommendation for the given customerNo
recommended_product('72231957',10,recommd_df)

In [ ]:
df_crossTabed['PERSON_ID'] = df_crossTabed.PERSON_ID.astype(str)

In [ ]:
def recommended_product(CustomerNo,data,DMatrix,Nproducts):
    data_items = data.drop('PERSON_ID', 1)
    data_items2 = data_items.divide(magnitude, axis='index')
    user_index = data[data.PERSON_ID == CustomerNo].index.tolist()[0]
    rating = data_items2.loc[user_index]
    known_user_likes = rating[rating >0].index.values
    recommend_set = rating[rating==0].index.values
    user_rating_vector = data_items2.loc[user_index]
    
    score = DMatrix.dot(user_rating_vector).div(DMatrix.sum(axis=1))
    
    recommend_products=score[recommend_set].nlargest(Nproducts)
    
    return recommend_products

In [ ]:
recommended_product('72231957',df_crossTabed,data_matrix,10)

In [ ]:
test_Old

In [ ]:
recommended_product('2726379162',df_crossTabed,data_matrix,5)

In [ ]:
SIV123                            0.140681
DM_P2                             0.117106
AB_CCARDS                         0.076253
AB_CWN                            0.052254
AB_AMF                            0.048927
ACCOPENING                        0.040577
PAY_DOM_BTB                       0.040577
PAY_DOM_URG                       0.040577
PAY_INT                           0.040577
PAY_DOM                           0.032169


In [ ]:
data_processed1

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
#X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
X=np.array(df_processed1)
nbrs = NearestNeighbors(n_neighbors=4, algorithm='auto').fit(X)
distances, indices = nbrs.kneighbors(X)
indices

In [ ]:
#Implementation with KDTree
from sklearn.neighbors import KDTree
import numpy as np
X=np.array(df_processed1)
kdt = KDTree(X, leaf_size=5, metric='euclidean')
kdt.query(X, k=4, return_distance=True)

In [ ]:
dist, ind = kdt.query(X[:384], k=3)  
print(ind)  # indices of 3 closest neighbors
print(dist)  # distances to 3 closest neighbors

In [ ]:
ind

In [ ]:
indList = ind.tolist()

In [ ]:
indList[0]

In [ ]:
simCustIndex = df.loc[indList[0]]

In [ ]:
simCustIdList = simCustIndex['PERSON_ID'].tolist()
simCustIdList

In [ ]:
 
custProddf=df_crossTabed.query("PERSON_ID in @simCustIdList")
custProddf

In [ ]:

sklearn.metrics.pairwise.pairwise_distances
pairwise_distances(X, Y=None, metric='euclidean', n_jobs=1, **kwds)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 10, random_state = 0)
clusters = kmeans.fit_predict(data_processed1)
kmeans.cluster_centers_

In [ ]:
print(kmeans.cluster_centers_.shape)

In [ ]:
scores =kmeans.score(data_processed1) 

In [ ]:
scores

In [ ]:

 ->single value /number:  